# Project Overview

Sparkify collects songs and user activity on their music streaming app. The analytics team wants to know what songs users are listening to. The data resides in a folder (event_data) of CSV files on user activity on the app.  

In this project, we create a Keyspace and tables in Apache Cassandra and build an ETL pipeline using Python, to answer three specific user questions.


## Part 1. Pre-Processing the Files

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part 2. Create Keyspaces in Apache Cassandra and import data

### Create Cluster

In [5]:
host = os.environ["HOSTADDR"]

from cassandra.cluster import Cluster
try: 
    cluster = Cluster([host]) 
    session = cluster.connect()
except Exception as e:
    print(e)

### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Query 1:  

### Retrieve the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
query1 = (
"""
CREATE TABLE IF NOT EXISTS user_activity \
(sessionId int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))
""")

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

query1 = ("""
INSERT INTO user_activity (sessionId, itemInSession, artist_name, song_title, song_length) \
VALUES (%s, %s, %s, %s, %s)
""")

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song_title, userId = line
        session.execute(query1, (int(sessionId), int(itemInSession), artist_name, song_title, float(length)))

In [10]:
#SELECT statement to verify the data was entered into the table
query1 = ("""
SELECT artist_name, song_title, song_length FROM user_activity \
WHERE sessionId = %s AND itemInSession = %s
""")

rows = session.execute(query1,(338,4))

for row in rows:
    print(row.artist_name,',', row.song_title,',', row.song_length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


## Query 2:

### Retrive only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# set userId and sessionId as a composite partition key
query2 = (
"""
CREATE TABLE IF NOT EXISTS user_listening_to_artist \
(userId int, sessionId int, itemInSession int, artist_name text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY ((userId, sessionId), itemInSession))
""")

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

query2 = ("""
INSERT INTO user_listening_to_artist (userId, sessionId, itemInSession, artist_name, song_title, user_first_name, user_last_name) \
VALUES (%s, %s, %s, %s, %s, %s, %s)
""")


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song_title, userId = line
        session.execute(query2, (int(userId), int(sessionId), int(itemInSession), artist_name, song_title, user_first_name, user_last_name))

In [13]:
# SELECT statement to verify the data was entered into the table
query2 = ("""
SELECT artist_name, song_title, user_first_name, user_last_name FROM user_listening_to_artist \
WHERE userid = %s AND sessionid = %s
""")

rows = session.execute(query2,(10,182))

for row in rows:
    print(row.artist_name,',', row.song_title,',', row.user_first_name +' '+ row.user_last_name)

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


## Query 3:  

### Retrive every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [14]:
query3 = (
"""
CREATE TABLE IF NOT EXISTS user_listening_to_song \
(song_title text, userId int, user_first_name text, user_last_name text, PRIMARY KEY (song_title, userId))
""")

try:
    session.execute(query3)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

query3 = ("""
INSERT INTO user_listening_to_song (song_title, userId, user_first_name, user_last_name) \
VALUES (%s, %s, %s, %s)
""")


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_first_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song_title, userId = line
        session.execute(query3, (song_title, int(userId), user_first_name, user_last_name))

In [16]:
# SELECT statement to verify the data was entered into the table
query3 = ("""
SELECT user_first_name, user_last_name FROM user_listening_to_song \
WHERE song_title = %s
""")

rows = session.execute(query3,('All Hands Against His Own', ))

for row in rows:
    print(row.user_first_name +' '+ row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table user_activity"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_listening_to_artist"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_listening_to_song"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "DROP KEYSPACE sparkify"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

errors={'52.151.117.78:9042': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=52.151.117.78:9042


### Close the session and cluster connection

In [19]:
session.shutdown()
cluster.shutdown()

In [20]:
#Some random debug queries
query = "SELECT * FROM system_schema.keyspaces"
#query = "SELECT COUNT(*) FROM user_activity"
#rows = session.execute(query)

for row in rows:
    print(row)